In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/FB_Player_Images

Mounted at /content/drive
[Errno 2] No such file or directory: '/content/drive/MyDrive/FB_Player_Images'
/content


In [ ]:
import os
import glob as glob
import matplotlib.pyplot as plt
import random
import cv2
from PIL import Image
import numpy

In [ ]:
# Function to convert bounding boxes in YOLO format to xmin, ymin, xmax, ymax.
def yolo2bbox(bboxes):
    xmin, ymin = bboxes[0]-bboxes[2]/2, bboxes[1]-bboxes[3]/2
    xmax, ymax = bboxes[0]+bboxes[2]/2, bboxes[1]+bboxes[3]/2
    return xmin, ymin, xmax, ymax

In [ ]:
def crop_image(image, bboxes, dest_dir, save_num, i):
    # Need the image height and width to denormalize
    # the bounding box coordinates
    h, w, _ = image.shape
    for box_num, box in enumerate(bboxes):
        x1, y1, x2, y2 = yolo2bbox(box)
        # Denormalize the coordinates.
        xmin = int(x1*w)
        ymin = int(y1*h)
        xmax = int(x2*w)
        ymax = int(y2*h)

        im = Image.fromarray(numpy.uint8(image))
        cropped_im = im.crop((xmin, ymin, xmax, ymax))
        # cropped_im = cropped_im.resize((30,15))
        isExist = os.path.exists(dest_dir + str(i))
        if not isExist:
            os.makedirs(dest_dir + str(i))
        cropped_im.save( dest_dir + str(i) + '/' + str(save_num) + '.jpg')
        save_num += 1
        # print(cropped_im.size)
    return save_num

In [ ]:
# Function to crop players from images.
def crop(image_paths, label_paths, num_samples, dest_dir):
    all_images = []
    all_images.extend(glob.glob(image_paths+'/*.jpg'))
    all_images.extend(glob.glob(image_paths+'/*.JPG'))
    all_labels = glob.glob(label_paths)
    
    all_images.sort()
    all_labels.sort()
    if num_samples == float("inf"):
      num_images = len(all_images)
    else:
      num_images = num_samples
    print(all_images)
    # save_num = 100
    for i in range(num_images):
        print(i)
        image = image = cv2.cvtColor(cv2.imread(all_images[i]), cv2.COLOR_BGR2RGB)
        with open(all_labels[i], 'r') as f:
            bboxes = []
            labels = []
            label_lines = f.readlines()
            for label_line in label_lines:
                label = label_line[0]
                if label == '2': #check if label == 'player'
                  bbox_string = label_line[2:]
                  x_c, y_c, w, h = bbox_string.split(' ')
                  x_c = float(x_c)
                  y_c = float(y_c)
                  w = float(w)
                  h = float(h)
                  bboxes.append([x_c, y_c, w, h])
                  labels.append(label)
        save_num = 100
        save_num = crop_image(image, bboxes,dest_dir, save_num, i)
        

In [ ]:
from matplotlib import numpy
def crop_image_train(image, bboxes, dest_dir, save_num):
    # Need the image height and width to denormalize
    # the bounding box coordinates
    h, w, _ = image.shape
    for box_num, box in enumerate(bboxes):
        x1, y1, x2, y2 = yolo2bbox(box)
        # Denormalize the coordinates.
        xmin = int(x1*w)
        ymin = int(y1*h)
        xmax = int(x2*w)
        ymax = int(y2*h)

        im = Image.fromarray(numpy.uint8(image))
        cropped_im = im.crop((xmin, ymin, xmax, ymax))
        cropped_im.save( dest_dir + str(save_num) + '.jpg')
        save_num += 1
        # print(cropped_im.size)
    return save_num

In [ ]:
# Function to crop players from train images.
def crop_train(image_paths, label_paths, num_samples, dest_dir):
    all_images = []
    all_images.extend(glob.glob(image_paths+'/*.jpg'))
    all_images.extend(glob.glob(image_paths+'/*.JPG'))
    all_labels = glob.glob(label_paths)
    
    all_images.sort()
    all_labels.sort()
    if num_samples == float("inf"):
      num_images = len(all_images)
    else:
      num_images = num_samples
    print(all_images)
    save_num = 0
    for i in range(num_images):
        print(i)
        image = cv2.cvtColor(cv2.imread(all_images[i]), cv2.COLOR_BGR2RGB)
        with open(all_labels[i], 'r') as f:
            bboxes = []
            labels = []
            label_lines = f.readlines()
            for label_line in label_lines:
                label = label_line[0]
                if label == '2': #check if label == 'player'
                  bbox_string = label_line[2:]
                  x_c, y_c, w, h = bbox_string.split(' ')
                  x_c = float(x_c)
                  y_c = float(y_c)
                  w = float(w)
                  h = float(h)
                  bboxes.append([x_c, y_c, w, h])
                  labels.append(label)
        save_num = crop_image(image, bboxes,dest_dir, save_num)
        

In [ ]:
# cropping all training images
crop_train(
    image_paths='/content/drive/MyDrive/yolov7-main/football-players-detection-2/train/images/' ,
    label_paths='/content/drive/MyDrive/yolov7-main/football-players-detection-2/train/labels/*.txt',
    num_samples= float("inf"), #inf takes all the pictures float("inf")
    dest_dir = '/content/drive/MyDrive/Deep Learning Project/FB_Player_Images_real/'
)

In [ ]:
#crop 5 validation images, we will label them, to check validation accuracy
crop(
    image_paths='/content/drive/MyDrive/Deep Learning Project/yolov7-main/football-players-detection-2/valid/images/' ,
    label_paths='/content/drive/MyDrive/Deep Learning Project/yolov7-main/football-players-detection-2/valid/labels/*.txt',
    num_samples= 10, #inf takes all the pictures float("inf")
    dest_dir = '/content/drive/MyDrive/Deep Learning Project/FB_Player_Images_validation/'
)

['/content/drive/MyDrive/Deep Learning Project/yolov7-main/football-players-detection-2/valid/images/08fd33_3_1_png.rf.894a7edd6b726ed886b3777ea55d6beb.jpg', '/content/drive/MyDrive/Deep Learning Project/yolov7-main/football-players-detection-2/valid/images/08fd33_3_3_png.rf.7aaf5d38e77c064882cb7b0f80e7e156.jpg', '/content/drive/MyDrive/Deep Learning Project/yolov7-main/football-players-detection-2/valid/images/08fd33_9_8_png.rf.c4792b7740c5e581674385c8a2c149fe.jpg', '/content/drive/MyDrive/Deep Learning Project/yolov7-main/football-players-detection-2/valid/images/121364_7_9_png.rf.0b474ed35ea4cab1acb2fc14ef64c7cc.jpg', '/content/drive/MyDrive/Deep Learning Project/yolov7-main/football-players-detection-2/valid/images/121364_9_2_png.rf.4028ff624f963fe501d9bf2b563582ca.jpg', '/content/drive/MyDrive/Deep Learning Project/yolov7-main/football-players-detection-2/valid/images/121364_9_3_png.rf.942d5bdea8c598c9ed8967bc8b9860ae.jpg', '/content/drive/MyDrive/Deep Learning Project/yolov7-main

In [ ]:
#crop 3 test images, we will label them, to check test accuracy
crop(
    image_paths='/content/drive/MyDrive/Deep Learning Project/yolov7-main/football-players-detection-2/test/images/' ,
    label_paths='/content/drive/MyDrive/Deep Learning Project/yolov7-main/football-players-detection-2/test/labels/*.txt',
    num_samples= 10, #inf takes all the pictures float("inf")
    dest_dir = '/content/drive/MyDrive/Deep Learning Project/FB_Player_Images_test/'
)

['/content/drive/MyDrive/Deep Learning Project/yolov7-main/football-players-detection-2/test/images/40cd38_7_6_png.rf.dc7957fcce915bc48188f18a7b19455b.jpg', '/content/drive/MyDrive/Deep Learning Project/yolov7-main/football-players-detection-2/test/images/42ba34_1_5_png.rf.927d602c034ce2997f58f62a7c1245ba.jpg', '/content/drive/MyDrive/Deep Learning Project/yolov7-main/football-players-detection-2/test/images/42ba34_5_5_png.rf.f691b18c36a92663c7241c99d4202bcc.jpg', '/content/drive/MyDrive/Deep Learning Project/yolov7-main/football-players-detection-2/test/images/4b770a_1_4_png.rf.8a0f6be9776a652d40938181f60b3a6c.jpg', '/content/drive/MyDrive/Deep Learning Project/yolov7-main/football-players-detection-2/test/images/4b770a_3_6_png.rf.f5d975605c1f73e1a95a1d8edc4ce5b1.jpg', '/content/drive/MyDrive/Deep Learning Project/yolov7-main/football-players-detection-2/test/images/573e61_1_9_png.rf.3e04feabb7fa88ed2691adcb5034e6ae.jpg', '/content/drive/MyDrive/Deep Learning Project/yolov7-main/footb